In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mlt
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML
import plotly.express as px
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
!pip install openpyxl

In [ ]:
epa_2020=pd.read_excel("../input/phase-ii-widsdatathon2022/epa/epa/Datathon_EPA_Air_Quality_Demographics_Meteorology_2020.xlsx")
epa_2019=pd.read_excel("../input/phase-ii-widsdatathon2022/epa/epa/Datathon_EPA_Air_Quality_Demographics_Meteorology_2019.xlsx")

# Downloaded the data. Examine it by using some basic analysis.
#### The EPA (Environmental Protection Agency) data provided for the competition has the following information captured CBSA wise. County and State wise data also available.
1. #### 	Demographic data (Fraction of people of colour, less than high school education, low income group and linguistically isolated
1. #### 	Weather related data : Temperature in Celsius, Wind Speed, Wind direction, Relative humidity
1. #### 	Air pollution data: Levels of Benzene ppm, Ozone ppm, Oxides of Nitrogen and Sulphur ppb, particulate matter 2.5m, lead , Carbon monoxide

In [ ]:
epa_2020.head(3)

In [ ]:
epa_2020.columns

In [ ]:
#Check the data types. Note from the output that the date is in datetime format and not string
epa_2020.dtypes

# Null Value Distribution across columns/Features

In [ ]:
nulldf= pd.DataFrame(epa_2020.isna().sum().sort_values(ascending=False))
nulldf['null']=nulldf.index
nulldf['count']=nulldf.iloc[:,:-1]
nulldf.reset_index(drop=True, inplace=True)
nulldf = nulldf.drop(nulldf.columns[[0]],axis = 1)
plt.title('Null Distribution Column-Wise')
ax = sns.barplot(y='null',x='count',data=nulldf.head(14))

for i in ax.containers:
  ax.bar_label(i)

print(nulldf['count'])

From the above, we note that LEAD_UG_PER_CUBIC_METER and BENZENE_PPBC have the highest null value distribution.

# Examining Demographic data.
## Plotting people of color and low income fraction across states

In [ ]:
# Plotting people of colour fraction and low income fraction together to see if there is any correlation 


df1=epa_2020.groupby('STATE',as_index=False)['PEOPLE_OF_COLOR_FRACTION','LOW_INCOME_FRACTION'].mean()

fig,(ax1)=plt.subplots(figsize=(20,8))

lns1=ax1.plot(df1.STATE,df1.PEOPLE_OF_COLOR_FRACTION,label='People of Colour Fraction')

ax2=ax1.twinx()

lns2=ax2.plot(df1.STATE,df1.LOW_INCOME_FRACTION,'r',label='Low Income Fraction')
lns=lns1+lns2
lab=[l.get_label()for l in lns]
ax1.legend(lns,lab,loc=0)
ax1.tick_params(labelrotation=90)


### Plotting Less than high school education and low income fraction across states

In [ ]:
# Plotting people of less than high school education fraction and low income fraction together to see if there is any correlation 


df1=epa_2020.groupby('STATE',as_index=False)['LESS_THAN_HS_ED_FRACTION','LOW_INCOME_FRACTION'].mean()

fig,(ax1)=plt.subplots(figsize=(20,8))

lns1=ax1.plot(df1.STATE,df1.LESS_THAN_HS_ED_FRACTION,label='Less than high school education')

ax2=ax1.twinx()

lns2=ax2.plot(df1.STATE,df1.LOW_INCOME_FRACTION,'r',label='Low Income Fraction')
lns=lns1+lns2
lab=[l.get_label()for l in lns]
ax1.legend(lns,lab,loc=0)
ax1.tick_params(labelrotation=90)


In [ ]:
#Distribution of linguistically isolated fraction across states
plt.figure(figsize=(20,6))

df1=epa_2020.groupby('STATE',as_index=False)['LINGUISTICALLY_ISOLATED_FRACTION'].mean()

ax=sns.barplot(x='STATE',y='LINGUISTICALLY_ISOLATED_FRACTION',data=df1.sort_values(by='LINGUISTICALLY_ISOLATED_FRACTION',ascending=False)     )     
plt.xticks(rotation=90)
ax.set_title("Distribution of liguistically isolated fraction(average) across states")

In [ ]:
#Plot as subplots, distribution of low income fraction, less than hs education fraction and people of colour fraction

# Plots of weather related data across states

In [ ]:
#Plotting mean temperature across all states in sorted order
plt.figure(figsize=(20,5))

df1=epa_2020.groupby('STATE',as_index=False)['TEMPERATURE_CELSIUS'].mean()


ax=sns.barplot(x='STATE',y='TEMPERATURE_CELSIUS',data=df1.sort_values(by='TEMPERATURE_CELSIUS',ascending=False)     )     
plt.xticks(rotation=90)
ax.set_title("Temperature in celsius across states")

In [ ]:
#Plot of average wind speed across states
plt.figure(figsize=(20,5))

df1=epa_2020.groupby('STATE',as_index=False)['WIND_SPEED_METERS_PER_SECOND'].mean()

ax=sns.barplot(x='STATE',y='WIND_SPEED_METERS_PER_SECOND',data=df1.sort_values(by='WIND_SPEED_METERS_PER_SECOND',ascending=False)     )     
plt.xticks(rotation=90)
ax.set_title("Wind Speed across states")

In [ ]:
#Plot of relative humidity across states, mean value
plt.figure(figsize=(20,5))

df1=epa_2020.groupby('STATE',as_index=False)['RELATIVE_HUMIDITY'].mean()

ax=sns.barplot(x='STATE',y='RELATIVE_HUMIDITY',data=df1.sort_values(by='RELATIVE_HUMIDITY',ascending=False)     )     
plt.xticks(rotation=90)
ax.set_title("Relative humidity across states")

# Plotting air pollutant data in subplots to reduce the length of the file

In [ ]:
#Start plotting air pollutants across states. This is to gather data and select only 6 top states to look at data month wise

fig,axes=plt.subplots(2,2,sharex=False,figsize=(30,14))

#Plotting ozone
df1=epa_2020.groupby('STATE',as_index=False)['OZONE_PPM'].mean()
sns.barplot(ax=axes[0,0],x='STATE',y='OZONE_PPM',data=df1.sort_values(by='OZONE_PPM',ascending=False))
axes[0,0].tick_params(labelrotation=90)
axes[0,0].set_title("OZONE_PPM",y=0.85,fontsize=16)

#Plotting Benzene ppbc
df1=epa_2020.groupby('STATE',as_index=False)['BENZENE_PPBC'].mean()
sns.barplot(ax=axes[0,1],x='STATE',y='BENZENE_PPBC',data=df1.sort_values(by='BENZENE_PPBC',ascending=False))
axes[0,1].tick_params(labelrotation=90)
axes[0,1].set_title("BENZENE_PPBC",y=0.75,fontsize=16)

#Plotting particulate matter 2.5micron
df1=epa_2020.groupby('STATE',as_index=False)['PM25_UG_PER_CUBIC_METER'].mean()
sns.barplot(ax=axes[1,0],x='STATE',y='PM25_UG_PER_CUBIC_METER',data=df1.sort_values(by='PM25_UG_PER_CUBIC_METER',ascending=False))
axes[1,0].tick_params(labelrotation=90)
axes[1,0].set_title("PM25_UG_PER_CUBIC_METER",y=0.75,fontsize=16)

#Plotting lead pcc
df1=epa_2020.groupby('STATE',as_index=False)['LEAD_UG_PER_CUBIC_METER'].mean()
sns.barplot(ax=axes[1,1],x='STATE',y='LEAD_UG_PER_CUBIC_METER',data=df1.sort_values(by='LEAD_UG_PER_CUBIC_METER',ascending=False))
axes[1,1].tick_params(labelrotation=90)
axes[1,1].set_title("LEAD_UG_PER_CUBIC_METER",y=0.75,fontsize=16)


In [ ]:
fig,axes=plt.subplots(3,1,sharex=False,figsize=(30,14))

#Start plotting air pollutants across states. This is to gather data and select only 6 top states to look at data month wise
#Plotting NO2
df1=epa_2020.groupby('STATE',as_index=False)['NO2_PPB'].mean()
sns.barplot(ax=axes[0],x='STATE',y='NO2_PPB',data=df1.sort_values(by='NO2_PPB',ascending=False))
axes[0].tick_params(labelrotation=90)
axes[0].set_title("NO2_PPB",y=0.75,fontsize=16)

#Plotting SO2
df1=epa_2020.groupby('STATE',as_index=False)['SO2_PPB'].mean()
sns.barplot(ax=axes[1],x='STATE',y='SO2_PPB',data=df1.sort_values(by='SO2_PPB',ascending=False))
axes[1].tick_params(labelrotation=90)
axes[1].set_title("SO2_PPB",y=0.75,fontsize=16)

#Plotting CO PPM
df1=epa_2020.groupby('STATE',as_index=False)['CO_PPM'].mean()
sns.barplot(ax=axes[2],x='STATE',y='CO_PPM',data=df1.sort_values(by='CO_PPM',ascending=False))
axes[2].tick_params(labelrotation=90)
axes[2].set_title("CO_PPM",y=0.75,fontsize=16)

# Heat maps of all numerical columns (demographic ,weather and air pollution )features across states data categorized as high medium and low

In [ ]:
data_epa_2019_mean = epa_2019.groupby(['STATE']).mean()
data_epa_2020_mean = epa_2020.groupby(['STATE']).mean()

In [ ]:
data_epa_2019_mean.drop(['LATITUDE','LONGITUDE'],axis=1,inplace=True)
data_epa_2020_mean.drop(['LATITUDE','LONGITUDE'],axis=1,inplace=True)

In [ ]:
#data_epa_2019_mean.drop(['LATITUDE','LONGITUDE'],axis=1,inplace=True)
#data_epa_2020_mean.drop(['LATITUDE','LONGITUDE'],axis=1,inplace=True)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
data_epa_2019_mean_scaled = scaler.fit_transform(data_epa_2019_mean.values)
data_epa_2020_mean_scaled = scaler.fit_transform(data_epa_2020_mean.values)

In [ ]:
data_epa_2019_mean_scaled_df = pd.DataFrame(data_epa_2019_mean_scaled,index=data_epa_2019_mean.index,columns = data_epa_2019_mean.columns)
data_epa_2020_mean_scaled_df = pd.DataFrame(data_epa_2020_mean_scaled,index=data_epa_2020_mean.index,columns = data_epa_2020_mean.columns)

In [ ]:
data_epa_2019_mean_scaled_df.columns

#### If values are below 0.3, replace it with 0, else keep the original value 

In [ ]:
data_epa_2019_mean_reduced = np.where(data_epa_2019_mean_scaled_df.iloc[0:52,0:15]<0.3,0,data_epa_2019_mean_scaled_df.iloc[0:52,0:15])
data_epa_2019_mean_reduced_df = pd.DataFrame(data_epa_2019_mean_reduced,index=data_epa_2019_mean.index,columns = data_epa_2019_mean_scaled_df.columns)

data_epa_2020_mean_reduced = np.where(data_epa_2020_mean_scaled_df.iloc[0:52,0:15]<0.3,0,data_epa_2020_mean_scaled_df.iloc[0:52,0:15])
data_epa_2020_mean_reduced_df = pd.DataFrame(data_epa_2020_mean_reduced,index=data_epa_2020_mean.index,columns = data_epa_2020_mean_scaled_df.columns)

#### If values in between 0.3 and 0.7, replace it with 0.5 , else keep the original value

In [ ]:
data_epa_2019_mean_reduced = np.where((data_epa_2019_mean_reduced_df.iloc[0:52,0:15]>=0.3) & (data_epa_2019_mean_reduced_df.iloc[0:52,0:15]<0.7)  ,0.5,data_epa_2019_mean_reduced_df.iloc[0:52,0:15])
data_epa_2019_mean_reduced_df = pd.DataFrame(data_epa_2019_mean_reduced,index=data_epa_2019_mean_reduced_df.index,columns = data_epa_2019_mean_reduced_df.columns)

data_epa_2020_mean_reduced = np.where((data_epa_2020_mean_reduced_df.iloc[0:52,0:15]>=0.3) & (data_epa_2020_mean_reduced_df.iloc[0:52,0:15]<0.7)  ,0.5,data_epa_2020_mean_reduced_df.iloc[0:52,0:15])
data_epa_2020_mean_reduced_df = pd.DataFrame(data_epa_2020_mean_reduced,index=data_epa_2020_mean_reduced_df.index,columns = data_epa_2020_mean_reduced_df.columns)

#### If values above 0.7, replace it with 1, else keep the original value

In [ ]:
data_epa_2019_mean_reduced = np.where(data_epa_2019_mean_reduced_df.iloc[0:52,0:15]>=0.7 ,1,data_epa_2019_mean_reduced_df.iloc[0:52,0:15])
data_epa_2019_mean_reduced_df = pd.DataFrame(data_epa_2020_mean_reduced,index=data_epa_2020_mean.index,columns = data_epa_2020_mean_scaled_df.columns)

In [ ]:
data_epa_2019_mean_demographic = data_epa_2019_mean_reduced_df[['PEOPLE_OF_COLOR_FRACTION','LOW_INCOME_FRACTION','LINGUISTICALLY_ISOLATED_FRACTION','LESS_THAN_HS_ED_FRACTION']]
data_epa_2020_mean_demographic = data_epa_2020_mean_reduced_df[['PEOPLE_OF_COLOR_FRACTION','LOW_INCOME_FRACTION','LINGUISTICALLY_ISOLATED_FRACTION','LESS_THAN_HS_ED_FRACTION']]

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(25,25))

#plt.figure(figsize = (5,25))
sns.heatmap(data_epa_2019_mean_demographic,ax=ax1, annot = True)
sns.heatmap(data_epa_2020_mean_demographic,ax=ax2, annot = True)
ax1.set_title("2019 Demographic data",fontsize=20)
ax2.set_title("2020 Demographic data",fontsize=20)
plt.show()

In [ ]:
data_epa_2019_mean_weather_AP = data_epa_2019_mean_reduced_df.iloc[:,4:15]
data_epa_2020_mean_weather_AP = data_epa_2020_mean_reduced_df.iloc[:,4:15]

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(25,25))
#plt.figure(figsize = (5,25))
sns.heatmap(data_epa_2019_mean_weather_AP, ax=ax1,annot = True)
ax1.set_title("2019 Average Weather data",fontsize=20)
#plt.figure(figsize = (5,25))
ax2.set_title("2020 Average Weather data",fontsize=20)
sns.heatmap(data_epa_2020_mean_weather_AP, ax=ax2,annot = True)

In [ ]:
#Indexing is to get specific columns
data_epa_2019_demographic = epa_2019.iloc[:,6:10]
data_epa_2020_demographic = epa_2020.iloc[:,6:10]

# Linear Correlation matrix for all numerical data

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
#plt.figure(figsize = (5,5))
matrix = data_epa_2019_demographic.corr()
matrix2= data_epa_2020_demographic.corr()
sns.heatmap(matrix, ax=ax1,annot = True)
sns.heatmap(matrix2, ax=ax2,annot = True)
ax1.set_title("2019 Correlation for demographic data",fontsize=20)
ax2.set_title("2020 Correlation for demographic data",fontsize=20)



In [ ]:
data_epa_2019_weather_ap = epa_2019.iloc[:,11:22]
data_epa_2020_weather_ap = epa_2020.iloc[:,11:22]

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(25,5))

matrix = data_epa_2019_weather_ap.corr()
matrix2 = data_epa_2020_weather_ap.corr()
sns.heatmap(matrix, ax=ax1,annot = True)
sns.heatmap(matrix, ax=ax2,annot = True)
ax1.set_title("2019 Correlation for weather data",fontsize=20)
ax2.set_title("2020 Correlation for weather data",fontsize=20)




# Pairplots
#### From the heat map, we see correlation for temperature, ozone level, lead level. 

In [ ]:
column_pairplot=['TEMPERATURE_CELSIUS','RELATIVE_HUMIDITY','OZONE_PPM','LEAD_UG_PER_CUBIC_METER']
sns.pairplot(epa_2020,kind='scatter',vars=column_pairplot,dropna=True,height=4.5,aspect=1)

Pair plot for Particulate matter, oxides of Nitrogen, Benzene, carbon monoxide

In [ ]:
column_pairplot=['PM25_UG_PER_CUBIC_METER','NO2_PPB','CO_PPM','BENZENE_PPBC']
sns.pairplot(epa_2020,kind='scatter',vars=column_pairplot,dropna=True,height=4.5,aspect=1)

In [ ]:
column_pairplot=['LEAD_UG_PER_CUBIC_METER','SO2_PPB','CO_PPM','BENZENE_PPBC']
sns.pairplot(epa_2020,kind='scatter',vars=column_pairplot,dropna=True,height=4.5,aspect=1)

Pivot of average temperature and ozone values across months and states

In [ ]:
epa_2020['MONTH_NAME']=epa_2020.DATE.dt.month_name()
state_group=epa_2020.groupby('STATE',as_index=False)['OZONE_PPM','TEMPERATURE_CELSIUS'].mean()
state_selected=[]
state_selected=state_group.sort_values(by='OZONE_PPM',ascending=False).head(6).STATE

#We select only top 6 of the states which have highest ozone. We tabulate the average ozone and temperature for each month for the 6 states

epa_pivot=epa_2020[epa_2020.STATE.isin(state_selected)].pivot_table(index='MONTH_NAME',columns='STATE',values={'TEMPERATURE_CELSIUS','OZONE_PPM'},sort=True,aggfunc=np.mean)
print(epa_pivot)


Pivot table with temperature and ozone data averaged over months for each state